In [1]:
import oracledb
import pandas as pd

# Connection details
username = "system"  # default admin user
password = "system"  # change this
dsn = "localhost:1521/XE"  # Oracle XE default

# Create connection
try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Successfully connected to Oracle Database")
    
    # Create a cursor
    cursor = connection.cursor()
    
except oracledb.DatabaseError as e:
    print("Error connecting to Oracle:", e)

Successfully connected to Oracle Database


In [2]:
try:
    # Create Banks table with corrected parenthesis
    cursor.execute("""
    CREATE TABLE banks (
        bank_id NUMBER GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
        bank_name VARCHAR2(100) NOT NULL UNIQUE,
        established_date DATE,
        headquarters VARCHAR2(100)
    )
    """)
    
    # Create Reviews table with corrected syntax
    cursor.execute("""
    CREATE TABLE reviews (
        review_id NUMBER GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
        review_text CLOB,
        rating NUMBER(1) NOT NULL CHECK (rating BETWEEN 1 AND 5),
        review_date DATE NOT NULL,
        bank_id NUMBER NOT NULL,
        votes NUMBER DEFAULT 0,
        sentiment_score NUMBER(3,2),
        processed_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        CONSTRAINT fk_review_bank FOREIGN KEY (bank_id) REFERENCES banks(bank_id)
    )
    """)
    
    connection.commit()
    print("Tables created successfully")

except oracledb.DatabaseError as e:
    error_obj, = e.args
    print(f"Error creating tables: [Oracle Error {error_obj.code}] {error_obj.message}")
    connection.rollback()

Error creating tables: [Oracle Error 955] ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/


In [3]:
# Insert banks
banks = [
    ("Commercial Bank of Ethiopia", None, "Addis Ababa"),
    ("Bank of Abyssinia", None, "Addis Ababa"),
    ("Dashen Bank", None, "Addis Ababa"),
    ("Awash Bank", None, "Addis Ababa")
]

try:
    cursor.executemany("""
    INSERT INTO banks (bank_name, established_date, headquarters)
    VALUES (:1, :2, :3)
    """, banks)
    
    connection.commit()
    print(f"{cursor.rowcount} banks inserted successfully")
    
except oracledb.DatabaseError as e:
    print("Error inserting banks:", e)
    connection.rollback()

Error inserting banks: ORA-00001: unique constraint (SYSTEM.SYS_C008338) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/


In [4]:
import pandas as pd
import oracledb

try:
    # Load and prepare data
    df = pd.read_csv('../data/All_banks_reviews.csv')
    
    # Data cleaning and validation
    df = df.dropna(subset=['review', 'rating', 'date', 'bank', 'votes'])  # Remove rows with missing critical fields
    
    # Convert and validate data types
    df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert with error handling
    df = df.dropna(subset=['date'])  # Remove rows with invalid dates
    
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]  # Ensure valid ratings
    
    df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0).astype(int)
    
    print(f"Loaded {len(df)} valid reviews for processing")

    # Get bank_id mapping
    cursor.execute("SELECT bank_id, bank_name FROM banks")
    bank_mapping = {row[1]: row[0] for row in cursor.fetchall()}
    
    # Prepare review data for insertion with validation
    review_data = []
    skipped_rows = 0
    
    for _, row in df.iterrows():
        bank_name = row['bank']
        if bank_name not in bank_mapping:
            print(f"Warning: Bank '{bank_name}' not found in database, skipping review")
            skipped_rows += 1
            continue
            
        review_data.append((
            str(row['review'])[:4000],  # Truncate to 4000 chars to avoid CLOB issues
            int(row['rating']),
            row['date'].to_pydatetime(),
            bank_mapping[bank_name],
            int(row['votes'])
        ))
    
    print(f"Prepared {len(review_data)} reviews for insertion ({skipped_rows} skipped)")

    # Batch insert with error handling
    batch_size = 1000  # Adjust based on your needs
    total_inserted = 0
    
    for i in range(0, len(review_data), batch_size):
        batch = review_data[i:i + batch_size]
        try:
            cursor.executemany("""
                INSERT INTO reviews (review_text, rating, review_date, bank_id, votes)
                VALUES (:1, :2, :3, :4, :5)
                """, batch)
            connection.commit()
            total_inserted += len(batch)
            print(f"Inserted batch {i//batch_size + 1}: {len(batch)} records (Total: {total_inserted})")
        except oracledb.DatabaseError as e:
            error_obj, = e.args
            print(f"Error inserting batch {i//batch_size + 1}: [Oracle Error {error_obj.code}] {error_obj.message}")
            connection.rollback()
            # Optionally log failed batch to retry later
            continue

    print(f"Successfully inserted {total_inserted} reviews out of {len(review_data)} attempted")

except Exception as e:
    print(f"Unexpected error: {str(e)}")
    if 'connection' in locals():
        connection.rollback()
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()

Loaded 9033 valid reviews for processing
Prepared 9033 reviews for insertion (0 skipped)
Inserted batch 1: 1000 records (Total: 1000)
Inserted batch 2: 1000 records (Total: 2000)
Inserted batch 3: 1000 records (Total: 3000)
Inserted batch 4: 1000 records (Total: 4000)
Inserted batch 5: 1000 records (Total: 5000)
Inserted batch 6: 1000 records (Total: 6000)
Inserted batch 7: 1000 records (Total: 7000)
Inserted batch 8: 1000 records (Total: 8000)
Inserted batch 9: 1000 records (Total: 9000)
Inserted batch 10: 33 records (Total: 9033)
Successfully inserted 9033 reviews out of 9033 attempted
